In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import urllib.request as ur
from get_risc_v_software_type import search_software_url
from tqdm import tqdm

In [2]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")

url_and_repo_name = search_software_url()
url_and_repo_name

100%|███████████████████████████████████████| 22/22 [00:00<00:00, 244760.45it/s]


[['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Accelerated%20Libraries',
  'Accelerated Libraries'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Accelerated%20Libraries,%20Linux,%20macOS',
  'Accelerated Libraries, Linux, macOS'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Application%20Infrastructure,%20Simulators',
  'Application Infrastructure, Simulators'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=Bootloaders',
  'Bootloaders'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=BSD%20Distro',
  'BSD Distro'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=C%20Compilers%20and%20Libraries',
  'C Compilers and Libraries'],
 ['https://riscv.org/exchange/?_sft_exchange_category=software&_sfm_exchange_software_type=C%2

In [3]:
github_link = []
for url,repo_name in tqdm(url_and_repo_name):
    
    browser = webdriver.Chrome(options = chrome_options)
    browser.maximize_window()
    software_type_url = url

    browser.get(url)

    prev_height = browser.execute_script("return document.body.scrollHeight")

    # 웹페이지 맨 아래까지 무한 스크롤
    while True:
        # 스크롤을 화면 가장 아래로 내린다
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(2)

        # 현재 문서 높이를 가져와서 저장
        curr_height = browser.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = browser.execute_script("return document.body.scrollHeight")



    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'lxml')


    
    software_list = soup.find_all("div", "col-results span_1_of_2-results bottom-results-right")
    
    for sw in software_list:

        link_name = sw.find('a')['href']
        if "github.com" in link_name:
            github_link.append(link_name)

100%|███████████████████████████████████████████| 22/22 [09:17<00:00, 25.33s/it]


In [15]:
github_link.remove('https://github.com/kvm-riscv/kvmtool.git')

ValueError: list.remove(x): x not in list

In [16]:
github_link.remove('https://github.com/hex-five/multizone-secure-iot-stack')

ValueError: list.remove(x): x not in list

In [22]:
github_link.remove('https://github.com/eclipse/tcf.agent')

In [25]:
github_link.remove('https://github.com/mono/mono/pull/11593')

In [28]:
github_link.remove('https://github.com/llvm/llvm-project/tree/master/llvm/lib/Target/RISCV')

In [31]:
github_link.remove('https://github.com/phoenix-rtos/phoenix-rtos-doc/tree/master/quickstart/')

In [34]:
github_link.remove('https://github.com/RT-Thread')

In [37]:
github_link.remove('https://github.com/keystone-enclave')

In [40]:
github_link.remove('https://github.com/Penglai-Enclave')

In [41]:
for i in github_link:
    print(i)

https://github.com/xianyi/OpenBLAS/tree/risc-v
https://github.com/moepinet/libmoepgf.git
https://github.com/olofk/vidbo
https://github.com/riscv/opensbi
https://github.com/riscv/riscv-pk
https://github.com/luojia65/rustsbi
https://github.com/riscv/riscv-uefi-edk2-docs
https://github.com/freebsd/freebsd
https://github.com/theandrew168/bronzebeard
https://github.com/AbsInt/CompCert.git
https://github.com/riscv/riscv-gcc
https://github.com/riscv/riscv-glibc
https://github.com/riscv/riscv-musl
https://github.com/riscv/riscv-newlib
https://github.com/gnu-mcu-eclipse/openocd
https://github.com/riscv/riscv-openocd
https://github.com/bao-project/bao-hypervisor
https://github.com/kvm-riscv/linux.git
https://github.com/mit-pdos/rvirt
https://github.com/xvisor/xvisor.git
https://github.com/riscv/riscv-gnu-toolchain
https://github.com/riscv/riscv-linux
https://github.com/riscv/meta-riscv
https://github.com/Tencent/ncnn
https://github.com/eclipse/omr
https://github.com/riscv/riscv-go
https://github

In [47]:
df = pd.DataFrame(
        columns=[
            "title",
            "descript",
            "topic_keyword",
            "star",
            "fork",
        ]
    )

for link in tqdm(github_link):
    df_dic = {}
    dic_keys = ['title', 'descript', 'topic_keyword', 'star', 'fork']

    url = link

    html = ur.urlopen(url)
    soup = bs(html.read(), "html.parser")

    for k in dic_keys:
        df_dic[k] = []



    title = soup.find('strong','mr-2 flex-self-stretch').text.strip()

    description = soup.find('p','f4 my-3')
    if description is not None:
        descript = description.text.strip()
    else:
        descript = '[BLANK]'

    topic_keyword = soup.find_all('a',{'data-ga-click':"Topic, repository page"})
    if topic_keyword is not None:
        topic_keyword_list = []
        for tk in topic_keyword:
            topic_keyword_list.append(tk.text.strip())
    else:
        topic_keyword_list = []


    list = []
    for i in soup.find_all('div','mt-2'):
        if i.find('strong') is not None:
            list.append(i.find('strong').text)

    df_dic['title'].append(title)
    df_dic['descript'].append(descript)
    df_dic['topic_keyword'].append(topic_keyword_list)
    df_dic['star'].append(list[0])
    df_dic['fork'].append(list[2])
    info = pd.DataFrame(df_dic)
    
    df = pd.concat([df,info], ignore_index=True)



100%|███████████████████████████████████████████| 66/66 [01:14<00:00,  1.13s/it]


In [49]:
df.to_csv('Learn_more_github_info.csv', encoding = 'utf-8')